In [604]:
# Standard library imports
from datetime import datetime
import os

# Third-party library imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm  # Visualize loop progress

# Set pandas display options for better data frame visualization
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Segreti
from dotenv import load_dotenv
load_dotenv()

# API
import requests
import json

| Gap-Up Screener |
|--|
TradingView Data (import)

In [605]:
# today's date string for file naming automation
today_date_str = datetime.now().strftime("_%Y-%m-%d.csv")
# today_date_str = '_2024-02-29.csv'

print(today_date_str)

_2024-03-06.csv


In [606]:
# concat base file name with today's date string
filename = f"trading_view_raw_data/tv_screen_gap-up{today_date_str}"
# read in trading view raw data
trading_view_raw_df = pd.read_csv(filename)

print(f'{len(trading_view_raw_df.index)} U.S. Stocks \n$(pre-market change) > $0.00 USD')
trading_view_raw_df.head(5)

1697 U.S. Stocks 
$(pre-market change) > $0.00 USD


,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Relative Volume at Time,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 day,Relative Volume 1 week,Relative Volume 1 month,New high 1 month,New high 1 month - Currency,New high 3 months,New high 3 months - Currency,New high 6 months,New high 6 months - Currency,New high 52 weeks,New high 52 weeks - Currency,New high All Time,New high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower"
0,MSFT,Microsoft Corporation,NASDAQ,2991865024204,USD,402.65,USD,404.48,USD,1.6500,USD,0.409785,0.454489,7318938817.36,26919177,44515133,139964,20043511.8,23220607.9,25412356.75555556,3.564297,1.77063671,1.45354449,405.846667,36.41683323,Technology Services,1.79896154,0.9529795,1.0291758,0.8906199,3.401715,3.425627,3.494110,2.72056179,1.32922035,0.80578435,1.06969419,1.36076495,0.42348812,0.11735635,420.82,USD,420.82,USD,420.82,USD,420.82,USD,420.82000,USD,402.85,USD,402.85,USD,402.85,USD,402.85,USD,402.85,USD,402.850,USD,414.25,USD,417.35,USD,417.35,USD,958891.40271493,USD,402.54800,402.401250,402.11602,401.572315,402.783991,402.094765,401.405539,402.652231,401.537350,400.422469
1,AAPL,Apple Inc.,NASDAQ,2626972696397,USD,170.12,USD,170.12,USD,1.3800,USD,0.811192,0.000000,15427863312.900002,95130377,176642456,584579,66996792.69999931,58859704.76666716,56051178.99999983,3.230751,1.66906114,1.51708484,170.593333,26.46958145,Electronic Technology,1.49335196,1.0467278,1.2041771,1.2891104,2.522859,3.033310,2.824503,2.05213039,0.88991179,0.4992566,0.75494684,1.51360234,0.68559221,0.20996954,191.05,USD,199.62,USD,199.62,USD,199.62,USD,199.62000,USD,170.22,USD,170.22,USD,170.22,USD,170.22,USD,170.22,USD,170.475,USD,172.04,USD,176.90,USD,180.53,USD,2380652.17391304,USD,170.10646,170.077787,170.07316,170.056969,170.201350,170.059595,169.917840,170.172883,170.033155,169.893427
2,NVDA,NVIDIA Corporation,NASDAQ,2149100036621.0005,USD,859.64,USD,864.16,USD,20.1596,USD,2.345121,0.525802,2399492500,52063600,113625575,913271,57962511.4,52115201.89999998,45803691.3777779,3.212762,3.2012168,3.80491577,851.593367,72.01776065,Electronic Technology,1.14108363,1.8023118,2.3419523,1.7245569,2.791819,2.878140,2.690679,1.66982475,0.67625101,0.40068698,0.66580538,0.87056351,0.50777842,0.16051256,876.95,USD,876.95,USD,876.95,USD,876.95,USD,876.95000,USD,860.00,USD,860.00,USD,860.00,USD,860.00,USD,860.00,USD,860.000,USD,860.97,USD,876.95,USD,876.95,USD,2058175.67567568,USD,859.27600,858.271250,854.95300,851.372692,861.148021,855.030130,848.912239,857.266897,850.172170,843.077443
3,AMZN,"Amazon.com, Inc.",NASDAQ,1808650704515.0005,USD,174.12,USD,174.88,USD,1.4100,

In [607]:
def categorize_market_cap(df):
    # convert 'Market capitalization' to numeric, coercing errors to NaN
    df['Market capitalization'] = pd.to_numeric(df['Market capitalization'], errors='coerce')
    # define conditions for market cap categories
    conditions = [
        # Titans
        (df['Market capitalization'] >= 200000000000),  # 200 billion and above
        #Large Cap
        (df['Market capitalization'] >= 10000000000) & (df['Market capitalization'] < 200000000000),  # 10 billion to 200 billion
        # Midlers
        (df['Market capitalization'] >= 2000000000) & (df['Market capitalization'] < 10000000000),  # 2 billion to 10 billion
        # Small Cap
        (df['Market capitalization'] >= 300000000) & (df['Market capitalization'] < 2000000000),  # 300 million to 2 billion
        # Micro Cap
        (df['Market capitalization'] > 50000000) & (df['Market capitalization'] < 300000000),  # 300 million and below (excluding 50 million and below)
        # Shrimp
        (df['Market capitalization'] <= 50000000)  # 50 million and below
    ]
    categories = ['Titans', 'Large caps', 'Midlers', 'Small caps', 'Micro caps', 'Shrimp']
    # use np.select to assign categories based on conditions
    df['marketCapType'] = np.select(conditions, categories, default='Undefined')
    
    return df


In [608]:
# EXECUTE
trading_view_raw_df = categorize_market_cap(trading_view_raw_df)
trading_view_raw_df.head(5)

,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Relative Volume at Time,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 day,Relative Volume 1 week,Relative Volume 1 month,New high 1 month,New high 1 month - Currency,New high 3 months,New high 3 months - Currency,New high 6 months,New high 6 months - Currency,New high 52 weeks,New high 52 weeks - Currency,New high All Time,New high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower",marketCapType
0,MSFT,Microsoft Corporation,NASDAQ,2.991865e+12,USD,402.65,USD,404.48,USD,1.6500,USD,0.409785,0.454489,7318938817.36,26919177,44515133,139964,20043511.8,23220607.9,25412356.75555556,3.564297,1.77063671,1.45354449,405.846667,36.41683323,Technology Services,1.79896154,0.9529795,1.0291758,0.8906199,3.401715,3.425627,3.494110,2.72056179,1.32922035,0.80578435,1.06969419,1.36076495,0.42348812,0.11735635,420.82,USD,420.82,USD,420.82,USD,420.82,USD,420.82000,USD,402.85,USD,402.85,USD,402.85,USD,402.85,USD,402.85,USD,402.850,USD,414.25,USD,417.35,USD,417.35,USD,958891.40271493,USD,402.54800,402.401250,402.11602,401.572315,402.783991,402.094765,401.405539,402.652231,401.537350,400.422469,Titans
1,AAPL,Apple Inc.,NASDAQ,2.626973e+12,USD,170.12,USD,170.12,USD,1.3800,USD,0.811192,0.000000,15427863312.900002,95130377,176642456,584579,66996792.69999931,58859704.76666716,56051178.99999983,3.230751,1.66906114,1.51708484,170.593333,26.46958145,Electronic Technology,1.49335196,1.0467278,1.2041771,1.2891104,2.522859,3.033310,2.824503,2.05213039,0.88991179,0.4992566,0.75494684,1.51360234,0.68559221,0.20996954,191.05,USD,199.62,USD,199.62,USD,199.62,USD,199.62000,USD,170.22,USD,170.22,USD,170.22,USD,170.22,USD,170.22,USD,170.475,USD,172.04,USD,176.90,USD,180.53,USD,2380652.17391304,USD,170.10646,170.077787,170.07316,170.056969,170.201350,170.059595,169.917840,170.172883,170.033155,169.893427,Titans
2,NVDA,NVIDIA Corporation,NASDAQ,2.149100e+12,USD,859.64,USD,864.16,USD,20.1596,USD,2.345121,0.525802,2399492500,52063600,113625575,913271,57962511.4,52115201.89999998,45803691.3777779,3.212762,3.2012168,3.80491577,851.593367,72.01776065,Electronic Technology,1.14108363,1.8023118,2.3419523,1.7245569,2.791819,2.878140,2.690679,1.66982475,0.67625101,0.40068698,0.66580538,0.87056351,0.50777842,0.16051256,876.95,USD,876.95,USD,876.95,USD,876.95,USD,876.95000,USD,860.00,USD,860.00,USD,860.00,USD,860.00,USD,860.00,USD,860.000,USD,860.97,USD,876.95,USD,876.95,USD,2058175.67567568,USD,859.27600,858.271250,854.95300,851.372692,861.148021,855.030130,848.912239,857.266897,850.172170,843.077443,Titans
3,AMZN,"Amazon.com, Inc.",NASDAQ,1.808651e+12,USD,174.12,U

In [609]:
# drop NaNs
tv_df = trading_view_raw_df.dropna()
print(f'NaNs = {tv_df.isna().sum().sum()}')

# confirm no NaNs
if tv_df.isna().sum().sum() == 0:
    nan_count = tv_df.isna().sum().sum()
    print(f"There are zero {nan_count} NaNs in DataFrame! Nice work!")
else:
    print(f"There are {tv_df.isna().sum().sum()} NaNs in DataFrame, check your code dumb dumb!")

NaNs = 0
There are zero 0 NaNs in DataFrame! Nice work!


In [610]:
# view the raw counts and percentages of each market cap type
market_cap_type_df = trading_view_raw_df['marketCapType'].value_counts().reset_index()
market_cap_type_df.columns = ['marketCapType', 'count']  # Rename columns for clarity
# calculate percentage -> add new columne to dataframe
market_cap_type_df['percentage'] = (market_cap_type_df['count'] / market_cap_type_df['count'].sum()) * 100

print(f'There are still the same {len(trading_view_raw_df.index)} U.S. Stocks in the dataframe \n$(pre-market change) > $0.00 USD')
market_cap_type_df

There are still the same 1697 U.S. Stocks in the dataframe 
$(pre-market change) > $0.00 USD


,marketCapType,count,percentage
0,Large caps,409,24.101355
1,Small caps,374,22.038892
2,Midlers,365,21.508544
3,Micro caps,250,14.731880
4,Shrimp,247,14.555097
5,Titans,37,2.180318
6,Undefined,15,0.883913


In [611]:
# print length of df
print(f'{len(tv_df.index)} U.S. Stocks \n$(pre-market change) > $0.00 USD')
# print sample head
tv_df.sample(5)

1682 U.S. Stocks 
$(pre-market change) > $0.00 USD


,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Relative Volume at Time,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 day,Relative Volume 1 week,Relative Volume 1 month,New high 1 month,New high 1 month - Currency,New high 3 months,New high 3 months - Currency,New high 6 months,New high 6 months - Currency,New high 52 weeks,New high 52 weeks - Currency,New high All Time,New high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower",marketCapType
800,HBM,Hudbay Minerals Inc.,NYSE,2.100693e+09,USD,5.99,USD,5.99,USD,0.13,USD,2.170284,0.000000,349956516,1972069,5432526,4200,2654937.5,2061881.4,2028905.94444444,3.781513,2.89077243,3.14588761,6.038333,31.92212494,Non-Energy Minerals,1.01403301,2.0097198,1.8624197,1.9631921,2.918783,2.962649,2.183370,1.79584327,1.01078113,0.56139762,0.98488903,0.7580533,0.66014005,0.16214465,6.18,USD,6.18,USD,6.18,USD,6.18,USD,28.890000,USD,6.00,USD,6.00,USD,6.00,USD,6.00,USD,6.00,USD,6.060,USD,6.175,USD,6.180,USD,6.180,USD,-,-,5.9930,5.990375,5.98460,5.977154,5.998193,5.984330,5.970467,6.030520,5.990150,5.949780,Midlers
991,GSM,Ferroglobe PLC,NASDAQ,8.134616e+08,USD,4.34,USD,4.35,USD,0.01,USD,0.230415,0.230415,112168377.828,2113384,4494201,699,2517251.2,1869497.53333333,1445813.06666667,3.472222,3.88553842,4.26996998,4.373333,9.46358482,Non-Energy Minerals,0.9850226,0.82664675,1.0151511,1.9972764,1.548935,4.781833,6.364780,3.498942,1.54183787,0.92931128,0.90270909,0.83921874,0.65991321,0.24462919,5.44,USD,6.78,USD,6.78,USD,6.78,USD,25.670000,USD,4.35,USD,4.37,USD,4.37,USD,4.37,USD,4.37,USD,4.410,USD,4.460,USD,4.600,USD,4.670,USD,-,-,4.3320,4.335625,4.34626,4.360485,4.366643,4.346565,4.326487,4.409476,4.371065,4.332654,Small caps
668,CUZ,Cousins Properties Incorporated,NYSE,3.603097e+09,USD,23.74,USD,23.81,USD,0.30,USD,1.263690,0.294861,150493250.06399998,1723517,2679236,11302,1721352.2,1802759.4,1670438.84444444,3.758099,3.09568623,2.61566034,23.636667,43.495786,Finance,0.92305393,1.930423,1.3292392,1.249568,4.813523,7.528838,8.172705,5.25952604,2.69567075,1.09717908,0.79115351,1.05659667,0.34663458,0.17021358,24.02,USD,25.19,USD,25.19,USD,25.19,USD,115.050453,USD,23.79,USD,23.79,USD,23.79,USD,23.79,USD,23.79,USD,23.880,USD,24.020,USD,24.020,USD,24.020,USD,2632373.7704918,USD,23.7270,23.742500,23.69400,23.653462,23.802008,23.683490,23.564972,23.776752,23.666000,23.555248,Midlers
1333,VANI,"Vivani Medical, Inc.",NASDAQ,1.097039e+08,USD,2.15,USD,2.26,USD,0.09,USD,4.186047,5.116279,25972439.2263,1319449,4050299,59782,18430432.2,6171736.83333333,2086402.73333333,16.355140,44.48494381,14.47648117,2.260000,-,Health Technology,0.05730255,0.3

In [612]:
# drop undefined
tv_df = tv_df[tv_df['marketCapType'] != 'Undefined']

market_cap_type_df2 = tv_df['marketCapType'].value_counts().reset_index()
market_cap_type_df2.columns = ['marketCapType', 'count']  # Rename columns for clarity

# calculate percentage for each market cap type and add it to the DataFrame
market_cap_type_df2['percentage'] = (market_cap_type_df2['count'] / market_cap_type_df2['count'].sum()) * 100

market_cap_type_df2
print(market_cap_type_df2)


  marketCapType  count  percentage
0    Large caps    409   24.316290
1    Small caps    374   22.235434
2       Midlers    365   21.700357
3    Micro caps    250   14.863258
4        Shrimp    247   14.684899
5        Titans     37    2.199762


In [613]:
# 1/3 #### CONFIG DATA SECTION ####
criteria_config = {
    "Titans": {
        "pre_market_change_pct_threshold": 0.002,  # Reduced to 0.2% for Titans
        "float_shares_outstanding_threshold": 1000000000,  # Increased to 1 billion shares
        "relative_volume_threshold": 1.2,  # Slightly more inclusive
        "relative_volume_at_time_threshold": 0.03,  # More inclusive
        "pre_market_gap_percentage_threshold": 0.001,  # Reduced to 0.1%
        "pre_market_vmap_drawdown_threshold": 0.003  # .3% drawdown from VWAP
    },
    "Large caps": {
        "pre_market_change_pct_threshold": 0.005,  # Reduced to 0.5% for Large caps
        "float_shares_outstanding_threshold": 200000000,  # Increased to 200 million shares
        "relative_volume_threshold": 1.3,  # Slightly more inclusive
        "relative_volume_at_time_threshold": 0.04,  # More inclusive
        "pre_market_gap_percentage_threshold": 0.005,  # Reduced to 0.5%
        "pre_market_vmap_drawdown_threshold": 0.004  # .4% drawdown from VWAP
    },
    "Midlers": {
        "pre_market_change_pct_threshold": 0.02,  # 2% for Mid caps
        "float_shares_outstanding_threshold": 50000000,  # 50 million shares
        "relative_volume_threshold": 1.3,
        "relative_volume_at_time_threshold": .05,
        "pre_market_gap_percentage_threshold": .02,
        "pre_market_vmap_drawdown_threshold": 0.005  # .5% drawdown from VWAP
    },
    "Small caps": {
        "pre_market_change_pct_threshold": 0.03,  # 3% for Small caps
        "float_shares_outstanding_threshold": 20000000,  # 20 million shares
        "relative_volume_threshold": 1.2,
        "relative_volume_at_time_threshold": .05,
        "pre_market_gap_percentage_threshold": .03,
        "pre_market_vmap_drawdown_threshold": 0.006  # 6% drawdown from VWAP
    },
    "Micro caps": {
        "pre_market_change_pct_threshold": 0.04,  # 4% for Micro caps
        "float_shares_outstanding_threshold": 5000000,  # 5 million shares
        "relative_volume_threshold": 1.1,
        "relative_volume_at_time_threshold": .05,
        "pre_market_gap_percentage_threshold": .04,
        "pre_market_vmap_drawdown_threshold": 0.007  # 7% drawdown from VWAP
    },
    "Shrimp": {
        "pre_market_change_pct_threshold": 0.05,  # 5% for Shrimp
        "float_shares_outstanding_threshold": 1000000,  # 1 million shares
        "relative_volume_threshold": 1.0,
        "relative_volume_at_time_threshold": .05,
        "pre_market_gap_percentage_threshold": .05,
        "pre_market_vmap_drawdown_threshold": 0.008  # 8% drawdown from VWAP
    }
}

# 2/3 #### CORE FUNCTIONS ####
def filter_by_pre_market_change(df, change_pct_threshold):
    """Filter stocks by pre-market change percentage."""
    return df[df['Pre-market Change %'] >= change_pct_threshold]

def filter_by_float_shares(df, float_shares_threshold):
    """Filter stocks by float shares outstanding."""
    # create a copy to safely modify without affecting the original DataFrame
    modified_df = df.copy()
    modified_df['Float shares outstanding'] = pd.to_numeric(modified_df['Float shares outstanding'], errors='coerce')
    return modified_df[modified_df['Float shares outstanding'] <= float_shares_threshold]

def filter_by_relative_volume(df, relative_volume_threshold):
    """Filter stocks by relative volume."""
    df['Relative Volume 1 day'] = pd.to_numeric(df['Relative Volume 1 day'], errors='coerce')
    return df[df['Relative Volume 1 day'] >= relative_volume_threshold]

def filter_by_relative_volume_at_time(df, relative_volume_at_time_threshold):
    """Filter stocks by relative volume at a specific time."""
    # Convert 'Relative Volume at Time' to float
    df['Relative Volume at Time'] = pd.to_numeric(df['Relative Volume at Time'], errors='coerce')
    return df[df['Relative Volume at Time'] >= relative_volume_at_time_threshold]

def filter_by_pre_market_gap_percentage(df, pre_market_gap_percentage_threshold):
    """Filter stocks by pre-market gap percentage."""
    return df[df['Pre-market Gap %'] >= pre_market_gap_percentage_threshold]

def filter_by_price_near_vwap(df, pre_market_vmap_drawdown_threshold):
    """
    Filter stocks where the price is near the VWAP, considering the specified drawdown threshold.
    """
    df_filtered = df.copy()
    df_filtered['Price'] = pd.to_numeric(df_filtered['Price'], errors='coerce')
    df_filtered['Volume Weighted Average Price 1 day'] = pd.to_numeric(df_filtered['Volume Weighted Average Price 1 day'], errors='coerce')
    
    # Calculate the acceptable minimum price based on the VWAP drawdown threshold
    df_filtered['Min Price from VWAP'] = df_filtered['Volume Weighted Average Price 1 day'] * (1 - pre_market_vmap_drawdown_threshold)
    
    # Apply the filter
    df_filtered = df_filtered[df_filtered['Price'] >= df_filtered['Min Price from VWAP']]
    
    return df_filtered.drop(columns=['Min Price from VWAP'])

"""SPECIAL FILTERS"""
def filter_by_volatility(df):
    """Filter stocks based on recent volatility being higher than weekly and monthly averages."""
    # Ensure data types are correct; convert to numeric if necessary
    df['Volatility 1 day'] = pd.to_numeric(df['Volatility 1 day'], errors='coerce')
    df['Volatility 1 week'] = pd.to_numeric(df['Volatility 1 week'], errors='coerce')
    df['Volatility 1 month'] = pd.to_numeric(df['Volatility 1 month'], errors='coerce')
    filtered_df = df[
        (df['Volatility 1 day'] >= df['Volatility 1 week']) & 
        (df['Volatility 1 day'] >= df['Volatility 1 month'])]
    return filtered_df

# 3/3 #### WRAPPER FUNCTION ####
def screen_stocks_by_category(df, category):
    """Screen stocks based on their market cap category."""
    # 3a. Get the configuration for the specified category
    config = criteria_config.get(category, {})
    pre_market_change_pct_threshold = config.get("pre_market_change_pct_threshold", 0)
    float_shares_threshold = config.get("float_shares_outstanding_threshold", float('inf'))
    relative_volume_threshold = config.get("relative_volume_threshold", 0)
    relative_volume_at_time_threshold = config.get("relative_volume_at_time_threshold", 0)
    pre_market_gap_percentage_threshold = config.get("pre_market_gap_percentage_threshold", 0)
    pre_market_vmap_drawdown_threshold = config.get("pre_market_vmap_drawdown_threshold", 0)
    # 3b. Apply the filters
    filtered_df = filter_by_pre_market_change(df, pre_market_change_pct_threshold)
    filtered_df = filter_by_float_shares(filtered_df, float_shares_threshold)
    filtered_df = filter_by_relative_volume(filtered_df, relative_volume_threshold)
    filtered_df = filter_by_relative_volume_at_time(filtered_df, relative_volume_at_time_threshold)
    filtered_df = filter_by_volatility(filtered_df) # SPECIAL FILTER
    # filtered_df = filter_by_price_above_vwap(filtered_df) # SPECIAL FILTER
    filtered_df = filter_by_pre_market_gap_percentage(filtered_df, pre_market_gap_percentage_threshold)
    filtered_df = filter_by_price_near_vwap(filtered_df, pre_market_vmap_drawdown_threshold)
  
    # 3c. Return the filtered DataFrame
    return filtered_df

In [614]:
#EXECUTE
tv_L2_df = pd.DataFrame()
categories = tv_df['marketCapType'].unique()

for category in categories:
    category_df = tv_df[tv_df['marketCapType'] == category]
    screened_df = screen_stocks_by_category(category_df, category)
    tv_L2_df = pd.concat([tv_L2_df, screened_df])

# Display results
print(f'Original DataFrame length: {len(tv_df.index)}')
print(f'Number of rows removed: {len(tv_df.index) - len(tv_L2_df.index)}')
print(f'NEW DataFrame length: {len(tv_L2_df.index)}')

# filter in descending order by market cap and then by pre-market change percentage
tv_L2_df = tv_L2_df.sort_values(by=['Market capitalization', 'Pre-market Change %'], ascending=[False, False])
# reset index and drop
tv_L2_df = tv_L2_df.reset_index(drop=True)

Original DataFrame length: 1682
Number of rows removed: 1670
NEW DataFrame length: 12


In [615]:
# create a breakdown of the market cap types
market_cap_type_df3 = tv_L2_df['marketCapType'].value_counts().reset_index()
market_cap_type_df3.columns = ['marketCapType', 'count']  # Rename columns for clarity
# calculate percentage for each market cap type and add it to the DataFrame
market_cap_type_df3['percentage'] = (market_cap_type_df3['count'] / market_cap_type_df3['count'].sum()) * 100

print('Returned Gap-Up Market Cap Type Groups:')
market_cap_type_df3

Returned Gap-Up Market Cap Type Groups:


,marketCapType,count,percentage
0,Large caps,5,41.666667
1,Midlers,4,33.333333
2,Small caps,2,16.666667
3,Titans,1,8.333333


In [616]:
# market cap type groups
print(f"Market Cap Groups:\n{tv_L2_df['marketCapType'].unique()}\n")

# TradingView column names
tv_raw_columns_list = tv_L2_df.columns.tolist()
print(f'TradingView Column Names (raw data):\n{tv_raw_columns_list}\n')

# stocks returned from the pre-market gap up screener
print(f'{len(tv_L2_df.index)} = U.S. stocks returned from the pre-market gap up screener.')

# create a list of 'Symbols' from the tv_L2_df
gap_up_stocks_list = tv_L2_df['Symbol']
gap_up_stocks_list = gap_up_stocks_list.tolist()
print(gap_up_stocks_list)

# FILTERD DATAFRAME WITH GAP-UP STOCK TARGETS
tv_L2_df


Market Cap Groups:
['Titans' 'Large caps' 'Midlers' 'Small caps']

TradingView Column Names (raw data):
['Symbol', 'Description', 'Exchange', 'Market capitalization', 'Market capitalization - Currency', 'Price', 'Price - Currency', 'Pre-market Open', 'Pre-market Open - Currency', 'Pre-market Change', 'Pre-market Change - Currency', 'Pre-market Change %', 'Pre-market Gap %', 'Float shares outstanding', 'Volume 1 day', 'Volume 1 week', 'Pre-market Volume', 'Average Volume 10 days', 'Average Volume 30 days', 'Average Volume 90 days', 'Volatility 1 day', 'Volatility 1 week', 'Volatility 1 month', 'Volume Weighted Average Price 1 day', 'Price to earnings ratio', 'Sector', 'Relative Volume at Time', 'Beta 1 year', 'Beta 3 years', 'Beta 5 years', 'Relative Volume 1 minute', 'Relative Volume 5 minutes', 'Relative Volume 15 minutes', 'Relative Volume 30 minutes', 'Relative Volume 1 hour', 'Relative Volume 2 hours', 'Relative Volume 4 hours', 'Relative Volume 1 day', 'Relative Volume 1 week', 'R

,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Relative Volume at Time,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 day,Relative Volume 1 week,Relative Volume 1 month,New high 1 month,New high 1 month - Currency,New high 3 months,New high 3 months - Currency,New high 6 months,New high 6 months - Currency,New high 52 weeks,New high 52 weeks - Currency,New high All Time,New high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower",marketCapType
0,COST,Costco Wholesale Corporation,NASDAQ,3.372114e+11,USD,759.95,USD,764.81,USD,5.39,USD,0.709257,0.639516,4.416039e+08,2353351,4486705,7152,1870259.8,1924204.16666667,2055289.41111111,1.634298,1.439577,1.267966,762.040000,51.82454872,Retail Trade,1.440777,0.646755,1.0365967,0.7653925,3.965740,4.155118,3.959101,2.88393967,1.3260042,0.75912065,0.94496841,1.254680,0.4983195,0.15716356,769.27,USD,769.27,USD,769.27,USD,769.27,USD,769.2700,USD,760.6899,USD,760.6899,USD,760.6899,USD,760.6899,USD,760.6899,USD,765.3600,USD,769.2700,USD,769.2700,USD,769.2700,USD,766740.50632911,USD,760.204,759.813425,758.83772,759.516431,760.713309,758.902680,757.092051,764.206943,760.550985,756.895027,Titans
1,LRCX,Lam Research Corporation,NASDAQ,1.262653e+11,USD,963.10,USD,972.00,USD,13.20,USD,1.370574,0.924099,1.303891e+08,1308328,2209953,2869,914883.6,1123711.36666667,1014679.3,2.846938,2.393549,2.686010,964.666667,37.23593455,Producer Manufacturing,1.129054,1.6386241,1.7224767,1.5171801,2.370672,3.118617,7.212501,5.14443337,2.28528193,1.03282744,1.09206075,1.465169,0.45925903,0.1419295,993.86,USD,993.86,USD,993.86,USD,993.86,USD,993.8600,USD,964.0999,USD,964.0999,USD,964.0999,USD,964.0999,USD,964.0999,USD,964.0999,USD,979.0000,USD,993.8600,USD,993.8600,USD,1013180.58139535,USD,962.966,962.093750,959.82202,956.080746,964.375678,959.638255,954.900832,962.121713,956.141235,950.160757,Large caps
2,VEEV,Veeva Systems Inc.,NYSE,3.568751e+10,USD,221.64,USD,222.08,USD,0.44,USD,0.198520,0.198520,1.466126e+08,1646396,2871446,117,1204801.4,998142.33333333,1217116.34444444,2.970189,2.940872,2.282509,222.194533,68.87079734,Technology Services,1.174330,1.5782332,1.0053954,0.7411759,2.252674,6.553840,5.118202,2.58780803,1.31214752,0.59013974,0.63051315,1.450122,0.63111429,0.23688819,230.655,USD,230.655,USD,230.655,USD,230.655,USD,343.9600,USD,222.3400,USD,222.4400,USD,222.4400,USD,222.4400,USD,222.4400,USD,223.6500,USD,225.4626,USD,226.5000,USD,230.6550,USD,-,-,221.993,222.005000,221.64500,221.142692,222.337662,221.714250,221.090838,222.343263,221.298000,220.252737,Large caps
3,TTWO,"Take-Two Interactive Software, Inc.",NASDAQ,2.455421e+10,USD,143.94,USD

| Gap-Up Screener |
|--|
IEX Cloud API (GET Request)

In [617]:
"""
A.
THIS IS ALSO GOING TO BE YOUR NEWS, CURRENT NEWS (like almost real-time (within 15-30mins))
-try to get foreign news where people are awake
-translation (internationalization) feature would be both advanced as well as practical
-all info about these companies so we start to learn what we are wokring with today
-maybe some fundamentals although less important for gap up? unless gap up turns into swing trade

B.
-need a closeness list. some rating system. avoid if a stock barely misses one or two filter metrics.
-so it's two part, 1-the filter closeness boolean as well as well has how close this miss was in terms of percentage and like an aggregate or miss difference -> THEN scoring system?

C.
-you are going to need also a -> 'chart_tv_df (this will be for the TradingView data)

"""

"\nA.\nTHIS IS ALSO GOING TO BE YOUR NEWS, CURRENT NEWS (like almost real-time (within 15-30mins))\n-try to get foreign news where people are awake\n-translation (internationalization) feature would be both advanced as well as practical\n-all info about these companies so we start to learn what we are wokring with today\n-maybe some fundamentals although less important for gap up? unless gap up turns into swing trade\n\nB.\n-need a closeness list. some rating system. avoid if a stock barely misses one or two filter metrics.\n-so it's two part, 1-the filter closeness boolean as well as well has how close this miss was in terms of percentage and like an aggregate or miss difference -> THEN scoring system?\n\nC.\n-you are going to need also a -> 'chart_tv_df (this will be for the TradingView data)\n\n"

In [618]:
"""NEWS AND THAT INTERNATIONALIZATION SHIT GOES HERE"""
info_news_df = pd.DataFrame(tv_L2_df, columns=['Symbol', 'Exchange', 'marketCapType'])

info_news_df

,Symbol,Exchange,marketCapType
0,COST,NASDAQ,Titans
1,LRCX,NASDAQ,Large caps
2,VEEV,NYSE,Large caps
3,TTWO,NASDAQ,Large caps
4,FNV,NYSE,Large caps
5,ELF,NYSE,Large caps
6,MTN,NYSE,Midlers
7,AVAV,NASDAQ,Midlers
8,TCBI,NASDAQ,Midlers
9,SOVO,NASDAQ,Midlers


In [619]:
# # manual add list
# manual_add_list = ['DELL','PLTR','TSLA']
# # append list with manually added stocks
# gap_up_stocks_list.extend(manual_add_list)

# print(f'{manual_add_list} added manually\n')
# print(f'New list: {gap_up_stocks_list}')

In [620]:
#### IEX CLOUD API KEY -> VARIABLE ####
iex_api_key = os.getenv("IEX_API_KEY")

# set the list of symbols variable
symbols = ','.join(gap_up_stocks_list)

# SETTING THE DATE RANGE
"""
y - Example: range=1y -> returns (one year)
ytd - Example: range=ytd -> returns (year-to-date)
m - Example: range=6m returns (six months)
d - Example: range=5d returns (five days)
"""
data_range = '4m' # DATE RANGE (for historical stock price data)

url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbols}&types=chart&range={data_range}&token={iex_key}'

# Make the GET request
response = requests.get(url)

# Initialize a list to collect DataFrames for each symbol
dataframes_list = []

# Check if the request was successful
if response.status_code == 200:
    # Convert the response to JSON
    data = response.json()

    # Process and print the data with a progress bar
    for symbol in tqdm(gap_up_stocks_list, desc="Processing symbols"):
        if symbol in data:
            # Create a DataFrame for the current symbol's data
            symbol_df = pd.DataFrame(data[symbol]['chart'])
            # Add a 'Ticker' column to identify the data
            symbol_df['Ticker'] = symbol
            # Append the current symbol's DataFrame to the list
            dataframes_list.append(symbol_df)
            # print(f'Data for {symbol}:')
            # print(symbol_df.head(5))  # print the first few rows of the DataFrame for this symbol
else:
    print(f'Failed to fetch data: {response.status_code}')

# concatenate dataframes and create a combined list
historical_df = pd.concat(dataframes_list, ignore_index=True)


Processing symbols: 100%|██████████| 12/12 [00:00<00:00, 1359.47it/s]


In [621]:
# Set variables for historical data analysis
historical_length = len(historical_df.index)
unique_stocks = historical_df['Ticker'].nunique()
start_date = historical_df['date'].min()
end_date = historical_df['date'].max()

print(f"START date: {start_date}")
print(f"END date: {end_date}")
print(f'{historical_df["date"].nunique()} days of historical stock data.\n')
print(f"{historical_length} = Length of dataframe")
print(f"{unique_stocks} = Unique stocks in dataframe")

# Entries per stock
days_per_stock = historical_length / unique_stocks
print(f"{days_per_stock} = Date entries per stock (MATH check -> should be the same as the unique )\n")

# View the stocks
print(f'Unique stock ticker symbols: {gap_up_stocks_list}')
historical_cols_list = historical_df.columns.tolist()
print(f'Columns: {historical_cols_list}')

# convert df to .csv
historical_df.to_csv(f'historical_stock_data{today_date_str}.csv', index=False)

# Display the first few rows of the dataframe
historical_df.head()

START date: 2023-11-06
END date: 2024-03-05
82 days of historical stock data.

984 = Length of dataframe
12 = Unique stocks in dataframe
82.0 = Date entries per stock (MATH check -> should be the same as the unique )

Unique stock ticker symbols: ['COST', 'LRCX', 'VEEV', 'TTWO', 'FNV', 'ELF', 'MTN', 'AVAV', 'TCBI', 'SOVO', 'MCB', 'CSTR']
Columns: ['close', 'high', 'low', 'open', 'priceDate', 'symbol', 'volume', 'id', 'key', 'subkey', 'date', 'updated', 'changeOverTime', 'marketChangeOverTime', 'uOpen', 'uClose', 'uHigh', 'uLow', 'uVolume', 'fOpen', 'fClose', 'fHigh', 'fLow', 'fVolume', 'label', 'change', 'changePercent', 'Ticker']


,close,high,low,open,priceDate,symbol,volume,id,key,subkey,date,updated,changeOverTime,marketChangeOverTime,uOpen,uClose,uHigh,uLow,uVolume,fOpen,fClose,fHigh,fLow,fVolume,label,change,changePercent,Ticker
0,569.82,572.650,560.130,560.49,2023-11-06,COST,2056898,HISTORICAL_PRICES,COST,,2023-11-06,1704323772000,0.000000,0.000000,560.49,569.82,572.650,560.1300,2056898,535.842,544.762,547.467,535.498,2056898,"Nov 6, 23",0.00,0.0000,COST
1,571.27,573.890,569.558,572.50,2023-11-07,COST,1373229,HISTORICAL_PRICES,COST,,2023-11-07,1704323770000,0.002545,0.002545,572.50,571.27,573.890,569.5575,1373229,547.324,546.148,548.653,544.511,1373229,"Nov 7, 23",1.45,0.0025,COST
2,567.38,572.990,564.295,572.12,2023-11-08,COST,1343554,HISTORICAL_PRICES,COST,,2023-11-08,1704323781000,-0.004282,-0.004282,572.12,567.38,572.990,564.2950,1343554,546.960,542.429,547.792,539.480,1343554,"Nov 8, 23",-3.89,-0.0068,COST
3,563.27,569.000,562.010,569.00,2023-11-09,COST,1288443,HISTORICAL_PRICES,COST,,2023-11-09,1704323778000,-0.011495,-0.011495,569.00,563.27,569.000,562.0100,1288443,543.978,538.500,543.978,537.295,1288443,"Nov 9, 23",-4.11,-0.0072,COST
4,577.12,577.885,564.940,567.90,2023-11-10,COST,1909263,HISTORICAL_PRICES,COST,,2023-11-10,1704323779000,0.012811,0.012811,567.90,577.12,577.885,564.9400,1909263,542.926,551.741,552.472,540.096,1909263,"Nov 10, 23",13.85,0.0246,COST


In [622]:
#### DEV - DATA ORGANIZATION ####
# convert df to .csv
historical_df.to_csv(f'/Users/sudz4/Desktop/FINANCIAL-SYSTEMS/dark-trader/trading_view_raw_data/historical_stock_data{today_date_str}.csv', index=False)

In [623]:
# Ensure the dataframe is sorted by date if it's not already
historical_sorted_df = historical_df.sort_values(by='date')

# Select mandatory columns for a historical price bar chart + volume
chart_historical_df = historical_sorted_df[[
    'symbol',   # *MANDATORY
    'date',     # *MANDATORY
    'close',    # *MANDATORY
    'high',     # *MANDATORY
    'low',      # *MANDATORY
    'open',     # *MANDATORY
    'volume'    # *MANDATORY
]]

# Group by 'symbol' and get the last occurrence
last_date_df = chart_historical_df.groupby('symbol').last().reset_index()

last_date_df

,symbol,date,close,high,low,open,volume
0,AVAV,2024-03-05,167.15,171.7000,150.110,150.64,3307117
1,COST,2024-03-05,759.95,769.2700,756.900,768.42,2353351
2,CSTR,2024-03-05,19.29,19.4100,18.500,18.55,153529
3,ELF,2024-03-05,201.62,206.5000,192.530,206.07,2488741
4,FNV,2024-03-05,112.52,113.5700,110.290,110.47,1584081
5,LRCX,2024-03-05,963.10,979.0000,951.900,969.99,1308329
6,MCB,2024-03-05,40.01,40.2400,36.650,36.65,141526
7,MTN,2024-03-05,219.06,221.9100,215.740,218.22,548142
8,SOVO,2024-03-05,22.86,22.9800,22.730,22.75,6827514
9,TCBI,2024-03-05,61.08,61.1400,57.560,57.56,542367


In [624]:
# re-index the dataframe
chart_historical_df.reset_index(drop=True, inplace=True)
# view head of your main df
print(f'Length of historical data: {len(chart_historical_df.index)}')

Length of historical data: 984


In [625]:
"""
YOUR DATAFRAMES TO MOVE FORWARD WITH:
"""
#### INFO / NEWS DATAFRAME ####
print(f'1 - NEWS & STOCK INFO (trending news etc.):\n{info_news_df}\n')

#### IEX (API) - HISTORICAL DATA FRAME ####
print('2a - IEX HISTORICAL data (START):')
print(f'{chart_historical_df.head()}\n')
print('2b - IEX HISTORICAL data (END):')
print(f'{chart_historical_df.tail()}\n')

1 - NEWS & STOCK INFO (trending news etc.):
   Symbol Exchange marketCapType
0    COST   NASDAQ        Titans
1    LRCX   NASDAQ    Large caps
2    VEEV     NYSE    Large caps
3    TTWO   NASDAQ    Large caps
4     FNV     NYSE    Large caps
5     ELF     NYSE    Large caps
6     MTN     NYSE       Midlers
7    AVAV   NASDAQ       Midlers
8    TCBI   NASDAQ       Midlers
9    SOVO   NASDAQ       Midlers
10    MCB     NYSE    Small caps
11   CSTR   NASDAQ    Small caps

2a - IEX HISTORICAL data (START):
  symbol        date   close     high     low    open   volume
0   COST  2023-11-06  569.82  572.650  560.13  560.49  2056898
1   VEEV  2023-11-06  196.15  196.800  192.55  194.31   877162
2   TTWO  2023-11-06  134.37  139.171  132.17  139.00  1926710
3    FNV  2023-11-06  124.43  125.310  123.91  124.21   781139
4    ELF  2023-11-06  101.68  102.750   95.52   97.66  1985721

2b - IEX HISTORICAL data (END):
    symbol        date   close      high      low    open   volume
979   VEEV  20

In [626]:
#### TRADINGVIEW (IMPORTED) - DATAFRAME (TODAY'S DATA) ####
print('3 - TRADINGVIEW (IMPORTED) - DATAFRAME (TODAY):')
print(f'{len(tv_L2_df.index)} = U.S. stocks returned from the pre-market gap up screener.')
tv_L2_df

3 - TRADINGVIEW (IMPORTED) - DATAFRAME (TODAY):
12 = U.S. stocks returned from the pre-market gap up screener.


,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Relative Volume at Time,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 day,Relative Volume 1 week,Relative Volume 1 month,New high 1 month,New high 1 month - Currency,New high 3 months,New high 3 months - Currency,New high 6 months,New high 6 months - Currency,New high 52 weeks,New high 52 weeks - Currency,New high All Time,New high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower",marketCapType
0,COST,Costco Wholesale Corporation,NASDAQ,3.372114e+11,USD,759.95,USD,764.81,USD,5.39,USD,0.709257,0.639516,4.416039e+08,2353351,4486705,7152,1870259.8,1924204.16666667,2055289.41111111,1.634298,1.439577,1.267966,762.040000,51.82454872,Retail Trade,1.440777,0.646755,1.0365967,0.7653925,3.965740,4.155118,3.959101,2.88393967,1.3260042,0.75912065,0.94496841,1.254680,0.4983195,0.15716356,769.27,USD,769.27,USD,769.27,USD,769.27,USD,769.2700,USD,760.6899,USD,760.6899,USD,760.6899,USD,760.6899,USD,760.6899,USD,765.3600,USD,769.2700,USD,769.2700,USD,769.2700,USD,766740.50632911,USD,760.204,759.813425,758.83772,759.516431,760.713309,758.902680,757.092051,764.206943,760.550985,756.895027,Titans
1,LRCX,Lam Research Corporation,NASDAQ,1.262653e+11,USD,963.10,USD,972.00,USD,13.20,USD,1.370574,0.924099,1.303891e+08,1308328,2209953,2869,914883.6,1123711.36666667,1014679.3,2.846938,2.393549,2.686010,964.666667,37.23593455,Producer Manufacturing,1.129054,1.6386241,1.7224767,1.5171801,2.370672,3.118617,7.212501,5.14443337,2.28528193,1.03282744,1.09206075,1.465169,0.45925903,0.1419295,993.86,USD,993.86,USD,993.86,USD,993.86,USD,993.8600,USD,964.0999,USD,964.0999,USD,964.0999,USD,964.0999,USD,964.0999,USD,964.0999,USD,979.0000,USD,993.8600,USD,993.8600,USD,1013180.58139535,USD,962.966,962.093750,959.82202,956.080746,964.375678,959.638255,954.900832,962.121713,956.141235,950.160757,Large caps
2,VEEV,Veeva Systems Inc.,NYSE,3.568751e+10,USD,221.64,USD,222.08,USD,0.44,USD,0.198520,0.198520,1.466126e+08,1646396,2871446,117,1204801.4,998142.33333333,1217116.34444444,2.970189,2.940872,2.282509,222.194533,68.87079734,Technology Services,1.174330,1.5782332,1.0053954,0.7411759,2.252674,6.553840,5.118202,2.58780803,1.31214752,0.59013974,0.63051315,1.450122,0.63111429,0.23688819,230.655,USD,230.655,USD,230.655,USD,230.655,USD,343.9600,USD,222.3400,USD,222.4400,USD,222.4400,USD,222.4400,USD,222.4400,USD,223.6500,USD,225.4626,USD,226.5000,USD,230.6550,USD,-,-,221.993,222.005000,221.64500,221.142692,222.337662,221.714250,221.090838,222.343263,221.298000,220.252737,Large caps
3,TTWO,"Take-Two Interactive Software, Inc.",NASDAQ,2.455421e+10,USD,143.94,USD

In [627]:
"""PICK UP HERE WHERE YOU LEFT OFF"""

'PICK UP HERE WHERE YOU LEFT OFF'

# DELL FOCUS - Pre-market

In [628]:
# # $DELL
# dell_df = key_levels_df[key_levels_df['symbol'] == 'DELL'].copy()
# dell_df['date'] = pd.to_datetime(dell_df['date'])
# dell_df.set_index('date', inplace=True)
# dell_df.head(5)

In [629]:
# import mplfinance as mpf
# # candlestick chart
# # MANDATORY DATA = 'open', 'high', 'low', 'close', 'volume'

# """
# Part 1:    
# """
# # Part 1a: SMAs -> calculate
# dell_df['SMA20'] = dell_df['close'].rolling(window=20, min_periods=1).mean()
# dell_df['SMA50'] = dell_df['close'].rolling(window=50, min_periods=1).mean()
# # dell_df['SMA200'] = dell_df['close'].rolling(window=200, min_periods=1).mean()

# # Part 1b: VWAP -> calculate
# dell_df['VWAP'] = (dell_df['volume'] * (dell_df['high'] + dell_df['low'] + dell_df['close']) / 3).cumsum() / dell_df['volume'].cumsum()

# """   
# Part 2: 
# """
# add2plot = [
#     #### SMAs ####
#     mpf.make_addplot(dell_df['SMA20'], color='#7BAFD4', width=1.0, label='SMA 20'), # UNC blue
#     mpf.make_addplot(dell_df['SMA50'], color='#003087', width=1.0, label='SMA 50'), # DUKE blue
#     # mpf.make_addplot(dell_df['SMA200'], color='#041E42', width=1.0, label='SMA 200'), # HOYAS blue
#     #### VWAP ####
#     mpf.make_addplot(dell_df['VWAP'], color='#B31B1B', width=1.0, label='VWAP') # BIG RED
# ]

# """   
# Part 3: styling not iterative changes related to data (i.e., color, font, etc.)
# """
# # Use a dark theme style and set text color to white
# mpf_style = mpf.make_mpf_style(base_mpf_style='nightclouds', rc={'text.color': 'white'})

# # Plot the candlestick chart with the moving averages
# fig, axes = mpf.plot(dell_df,
#                      type='candle',
#                      addplot=add2plot,
#                      volume=True,
#                     #  style=mpf_style,
#                      style=s, 
#                      # WTF look at docs (it's not erroring out but going to default style just figure out what that is i think)
#                      figratio=(15, 8),
#                      title='$DELL - Historical Data (6 Months)',
#                      tight_layout=True,
#                      returnfig=True)

# # Add a watermark if needed
# fig.text(0.5, 0.5, 'DELL', fontsize=50, color='gray', alpha=0.5, ha='center', va='center', rotation=0, transform=fig.transFigure)

# plt.show()


In [630]:
# dell_df.head(5)

In [631]:
""" 
ISSUE: some weird shit with the style param
nbd tho ctfo. 

workaround =  is first successfully run with style=mpf_style, then swap it for style=s (dark mode - but common view like Ortex)  
the blue shit and mpf_style is actually cool but might confuse user (i.e., blue = red and white = green) with the mpf_style
experienced similar shit like this years ago, idunno will get to it.

"""

' \nISSUE: some weird shit with the style param\nnbd tho ctfo. \n\nworkaround =  is first successfully run with style=mpf_style, then swap it for style=s (dark mode - but common view like Ortex)  \nthe blue shit and mpf_style is actually cool but might confuse user (i.e., blue = red and white = green) with the mpf_style\nexperienced similar shit like this years ago, idunno will get to it.\n\n'

In [632]:
# you're going to need different charts, one for the pre-market, and at least another for market open
# I'd like market opens to be a live feed but really we might already be in trading view watching the pine script execute

# Trend lines

| END OF IN PROGRESS WORK - NOTES BELOW |
|--|

In [633]:
# but first start with getting lines on that chart with STRV
# let's get those key levels with an output on them and figure out your strategy
# figure out when fibonacci extensions get activated
# i see one view with a grid layout of the stock charts above or below them the summary with the key levels, current price and all that
# i also see a view where when looking at individiual stocks we got data all over that shit. idgaf if it's "confusing for the user"

# you have your key_levels_df
# so start next from there okay

#### </b> ####

# make title more symetrically bigger not just bigger for fucks sake 
# move price to the right side, keep vol on left
# see if you can get a marker or military like custom google font one of the ones you like
# better yet use that program you wrote for sn logos actually to create your shit
# could put that Sans Peur Systems somewhere at least watermark it, maybe?
# key levels find up to 6?, scoring strategy?, then base allotment of position (1/3, 1/5) conditionally?
# we could add RSI down there, fuck could add ichimoku, do whatever with all this data access now

NOTES -> SMA Legend

In [634]:
# # SMA20, SMA50, and SMA200 has to be in 'dell_df'
# legend_info = {
#     'Indicator': ['SMA20', 'SMA50', 'SMA200'],
#     'Color': ['blue', 'orange', 'green'],
#     'Description': ['20-day Moving Average', '50-day Moving Average', '200-day Moving Average']
# }

# legend_df = pd.DataFrame(legend_info)
# print(legend_df)


NOTES -> custom mplfinance chart setup example

In [635]:
# """    
# Custom Theme setup
# """

# dark_style = mpf.make_mpf_style(base_mpf_style='charles', 
#                                 rc={'axes.facecolor': 'black',
#                                     'figure.facecolor': 'darkslategrey',
#                                     'axes.grid': True,
#                                     'grid.color': 'gray',
#                                     'grid.linestyle': '--',
#                                     'grid.linewidth': 0.6,
#                                     'axes.titlesize': 16,
#                                     'axes.titleweight': 'bold',
#                                     'axes.labelsize': 12,
#                                     'axes.labelweight': 'bold',
#                                     'axes.labelcolor': 'white',
#                                     'axes.edgecolor': 'white',
#                                     'axes.linewidth': 2,
#                                     'xtick.color': 'white',
#                                     'xtick.labelsize': 10,
#                                     'ytick.color': 'white',
#                                     'ytick.labelsize': 10,
#                                     'figure.titlesize': 18,
#                                     'figure.titleweight': 'bold',
#                                     'legend.facecolor': 'grey',
#                                     'legend.edgecolor': 'white',
#                                     'legend.fontsize': 10,
#                                     'legend.title_fontsize': 12})

# mpf.plot(strv_df, 
#          type='candle', 
#          style=dark_style,  # Using the custom dark style
#          volume=True, 
#          title='STRV Candlestick Chart',
#          mav=(20, 50, 200),  # Adding moving averages of 20, 50, and 200 periods
#          figratio=(15, 8), 
#          tight_layout=True)

In [636]:
# i need a math checker for trading days vs calendar days (252/365)

Notes -> yahoo finance API exaple (free) *with limitations

In [637]:
# import yfinance as yf
# from datetime import datetime, timedelta

In [638]:
# # yahoo finance data download
# start = datetime.now() - timedelta(days=60)
# end = datetime.now()
# yahoo_df = yf.download(gap_up_stocks_list, start, end)
# yahoo_df = yahoo_df.loc[:,'Close']
# # view the yahoo data yahoo finance formatting multi-index (has a 60day max with standard API maneuvers)
# yahoo_df.head(10)